In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


ANN modeling SAMPLE

In [ ]:
#%% ANN modeling
import tensorflow as tf
from tensorflow import keras

def define_model():
    
    input_mnist = keras.Input(shape=(input_process.shape[1],))
    model1 = keras.layers.Dense(units = 2048)(input_mnist)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dropout(0.2)(model1)
    model1 = keras.layers.Dense(units = 1024)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dropout(0.2)(model1)
    model1 = keras.layers.Dense(units = 512)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dropout(0.2)(model1)
    model1 = keras.layers.Dense(units = 256)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dropout(0.2)(model1)
    model1 = keras.layers.Dense(units = 256)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dropout(0.2)(model1)
    model1 = keras.layers.Dense(units = 128)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dense(units = 128)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dense(units = 64)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)
    model1 = keras.layers.Dense(units = 32)(model1)
    model1 = keras.layers.Activation(activation = 'relu')(model1)    
    model1 = keras.layers.Dense(units = 1)(model1)
    model1 = keras.layers.Activation(activation = 'linear')(model1)       
    
    return keras.Model(inputs=input_mnist, outputs=model1)

NN_model = define_model()

opt = tf.keras.optimizers.Adam(learning_rate = 0.001)
NN_model.compile(optimizer = opt, loss = tf.keras.losses.MeanSquaredError())
    
NN_model.summary()    

history_callback = NN_model.fit(train_x, train_y, batch_size = 512, epochs = 80, shuffle=True, validation_data = (test_x, test_y))


data_process_1

In [ ]:
# dataset import
import pandas as pd

regionname = ["강원영동", "강원영서", "경남", "경북", "서울경기", "전남", "전북", "제주", "충남", "충북"]
filename = ["강수량", "습도", "평균기온", "풍속"]
dataset = []
for i in range(len(regionname)):
    dataset_tmp = []
    for j in range(len(filename)):
        dataset_tmp.append(pd.read_csv(regionname[i] + "/" + regionname[i] + " " + filename[j] + ".csv"))
    dataset.append(dataset_tmp)

datarowindex = [11,13,6,12]
datacolumnindex = [[2,3,4], [2,3,4], [0,2,3,4], [2,3,4]]

del i,j,dataset_tmp
#%%
data_output = [[] for i in range(4)]

for i in range(len(regionname)):
    for j in range(4):
        data_tmpj = []
        if(i==0):
            data_tmpk = dataset[i][j].iloc[datarowindex[j]:,datacolumnindex[j][0]].values
            data_tmpk = data_tmpk[0:241]
            data_tmpj.append(data_tmpk)
        for jj in range(1,len(datacolumnindex[j])):
            data_tmpk = dataset[i][j].iloc[datarowindex[j]:,datacolumnindex[j][jj]].values
            data_tmpk = data_tmpk[0:241]
            data_tmpj.append(data_tmpk)
            
        for ii in range(len(data_tmpj)):
            if(not (i==0 and ii==0)):
                data_tmpj[ii][0] = regionname[i] + " " + data_tmpj[ii][0]
            data_output[j].append(data_tmpj[ii])

for i in range(4):
    label = []
    for ii in range(len(data_output[i])):
        label.append(data_output[i][ii][0])
    data = []
    if(i==2):
        for ii in range(1,241):
            tmp = []
            for iii in range(len(data_output[i])):
                tmp.append(data_output[i][iii][ii])
            data.append(tmp)
    else:
        for ii in range(240,0,-1):
            tmp = []
            for iii in range(len(data_output[i])):
                tmp.append(data_output[i][iii][ii])
            data.append(tmp)
   
    
    RF_pd = pd.DataFrame(data, columns = label)
    RF_pd.to_csv("data" + filename[i] + ".csv", encoding='utf-8-sig', index = None)
        

del i,j,ii,jj,iii

data_process_2

In [ ]:
# dataset import
import pandas as pd

regionname = ["강원영동", "강원영서", "경남", "경북", "서울경기", "전남", "전북", "제주", "충남", "충북"]
regionname_process = ["서울특별시", "부산광역시", "대구광역시", "인천광역시", "광주광역시", "대전광역시", "울산광역시", 
                      "세종특별자치시", "경기도", "강원도", "충청북도", "충청남도", "전라북도", "전라남도", "경상북도", "경상남도", "제주도"]
regionname_index = [4,2,3,4,5,8,2,8,4,1,9,8,6,5,3,2,7]
month = [i for i in range(1,13)]

filename = ["강수량", "습도", "평균기온", "풍속"]

dataset = []
for i in range(len(filename)):
    dataset_tmp = pd.read_csv("data" + filename[i] + ".csv")
    dataset.append(dataset_tmp)

del i

#%% 
import numpy as np

label = [["지역별", "년", "월", "평균월강수량", "최다월강수량"],
         ["지역별", "년", "월", "평균습도", "최저습도"],
         ["지역별", "년", "월", "평균기온", "최저기온", "최고기온"],
         ["지역별", "년", "월", "평균풍속", "최대풍속"]]
count = [2,2,3,2]

dataset_output = [[] for i in range(4)]
data_region = [regionname_process[i%len(regionname_process)] for i in range(len(regionname_process)*dataset[0].shape[0])]
data_year = [(2000 + int(i/(len(regionname_process)*12))) for i in range(len(regionname_process)*dataset[0].shape[0])]
data_month = [month[int((i%(len(regionname_process)*12))/len(regionname_process))] for i in range(len(regionname_process)*dataset[0].shape[0])]

for i in range(len(filename)):
    for j in range(count[i]):
        data_tmp = []
        for k in range(dataset[i].shape[0]):
            row = k
            for m in range(len(regionname_index)):
                col = count[i] * regionname_index[m] + 1 + j
                data_tmp.append(dataset[i].iloc[row,col])
        dataset_output[i].append(data_tmp)
    
for i in range(len(filename)):
    dataset_tmp = np.c_[data_region, data_year, data_month]
    for j in range(len(label[i]) - 3):
        dataset_tmp = np.c_[dataset_tmp, dataset_output[i][j]]
    RF_pd = pd.DataFrame(dataset_tmp, columns = label[i])
    RF_pd.to_csv("data" + filename[i] + ".csv", encoding='utf-8-sig', index = None)